# Mapping electricity generation and trade to ecoinvent

To run this notebook, you must have the following environment variables set:

* BENTSO_DATA_DIR: Directory to cache data from ENTSO-E API

In addition, you need to have finished the notebook `ENTSO-E electricity data`.

In [163]:
import bw2data as bd
import bw2io as bi
import pandas as pd
import numpy as np
from collections import defaultdict
import bw_processing as bwp
import bentso
from entsoe.exceptions import NoMatchingDataError

In [2]:
assert bd.__version__ >= (4, 0, 'DEV11')
assert bentso.__version__ >= (0, 4)

In [3]:
bd.projects.set_current('GSA for archetypes')

## Find electricity generators in Europe

In [4]:
ei = bd.Database("ecoinvent 3.8 cutoff")
assert len(ei)

In [5]:
from bentso.constants import ENTSO_COUNTRIES, TRADE_PAIRS

In [6]:
def get_one(db_name, **kwargs):
    possibles = [act for act in bd.Database(db_name) if all(act.get(key) == value for key, value in kwargs.items())]
    if len(possibles) == 1:
        return possibles[0]
    else:
        raise ValueError(f"Couldn't get exactly one activity in database `{db_name}` for arguments {kwargs}")

In [7]:
def is_generation_exchange(exc):
    return (exc.input['unit'] == 'kilowatt hour' 
        and "import from" not in exc.input['name'] 
        and not exc.input['name'].startswith('market')
        and 'voltage transformation' not in exc.input['name']
    )

In [8]:
def is_trade_exchange(exc):
    return (exc.input['unit'] == 'kilowatt hour' 
        and "import from" in exc.input['name'] 
    )

In [9]:
NAMES = {
    'market for electricity, low voltage',
    'market for electricity, medium voltage',
    'market for electricity, high voltage',
}

TRANSFORMATION = {
    'electricity voltage transformation from medium to low voltage',
    'electricity voltage transformation from high to medium voltage',
}

In [10]:
markets = {
    (act['name'], act['location']): act
    for act in ei
    if act['name'] in NAMES.union(TRANSFORMATION)
    and act['location'] in ENTSO_COUNTRIES
}

In [11]:
found = defaultdict(set)

for country in ENTSO_COUNTRIES:
    market = markets[('market for electricity, high voltage', country)]
    for exc in market.technosphere():
        if is_trade_exchange(exc) and exc.input['name'].startswith('electricity, high voltage, import from'):
            found[country].add(exc.input['name'][-2:])

In [12]:
for country, lst in found.items():
    if lst.difference(TRADE_PAIRS.get(country, set())):
        print(country, lst.difference(TRADE_PAIRS.get(country, set())))

In [13]:
def get_generators(market):
    found = set()
    
    for act in ei:
        if act['name'] == market and act['location'] in ENTSO_COUNTRIES:
            for exc in act.technosphere():
                if is_generation_exchange(exc):
                    found.add(exc.input['name'])
                    
    return found

In [14]:
def get_scaling(country, act_name='electricity voltage transformation from high to medium voltage'):
    trans = markets[(act_name, country)]
    amount = sum(exc['amount'] for exc in trans.technosphere())
    assert 1 < amount < 1.25
    return amount

In [15]:
for country in ENTSO_COUNTRIES:
    print(country, get_scaling(country), get_scaling(country, 'electricity voltage transformation from medium to low voltage'))

CH 1.0062 1.0276
NL 1.00377404701174 1.01747560791302
MK 1.00910857670359 1.04185326219664
AT 1.00396869240528 1.01808622303526
MT 1.00503071204738 1.02391624781522
PT 1.00812196540363 1.03761463678944
SK 1.00343356121558 1.01574784947215
GR 1.00818035656772 1.03899586799491
BE 1.00370139222692 1.01673171811526
SI 1.00491340196632 1.02253575107447
IT 1.00501905180405 1.02340471648591
PL 1.00430743835793 1.01957900530224
EE 1.00694992560631 1.03127395498758
FI 1.00269857339598 1.01207896511049
NO 1.00572337921071 1.02600130953742
IE 1.00639279244748 1.02848269915284
DE 1.00441453794017 1.02055562344301
DK 1.00542069965637 1.02485855859503
HU 1.00615371244631 1.02807179358003
RS 1.01058139679409 1.04872102986472
RO 1.01045166498605 1.04901285503394
ME 1.00759987180707 1.03476080059219
BG 1.00731866170315 1.03404940677966
LU 1.00348553893375 1.01567080244515
HR 1.00829697146417 1.03808897071116
FR 1.00670440632908 1.03093659171436
GB 1.00680829804927 1.03157377466283
LV 1.00501217963228 1

In [16]:
hv = get_generators("market for electricity, high voltage")
hv

{'electricity production, deep geothermal',
 'electricity production, hard coal',
 'electricity production, hydro, pumped storage',
 'electricity production, hydro, reservoir, alpine region',
 'electricity production, hydro, reservoir, non-alpine region',
 'electricity production, hydro, run-of-river',
 'electricity production, lignite',
 'electricity production, natural gas, combined cycle power plant',
 'electricity production, natural gas, conventional power plant',
 'electricity production, nuclear, boiling water reactor',
 'electricity production, nuclear, pressure water reactor',
 'electricity production, nuclear, pressure water reactor, heavy water moderated',
 'electricity production, oil',
 'electricity production, peat',
 'electricity production, solar thermal parabolic trough, 50 MW',
 'electricity production, solar tower power plant, 20 MW',
 'electricity production, wind, 1-3MW turbine, offshore',
 'electricity production, wind, 1-3MW turbine, onshore',
 'electricity produ

In [17]:
mv = get_generators("market for electricity, medium voltage")
mv

{'electricity, from municipal waste incineration to generic market for electricity, medium voltage'}

In [18]:
lv = get_generators("market for electricity, low voltage")
lv

{'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
 'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
 'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
 'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
 'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
 'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
 'electricity production, photovoltaic, 3kWp slanted-roof 

In [19]:
all_generation = set.union(hv, mv, lv)

# Swiss specific fixes

In [106]:
def get_swissgrid_df(fp):
    df = pd.read_excel(fp, sheet_name="Zeitreihen0h15", usecols=[2], header=1)
    df = df.groupby(df.index // 4).sum()
    return df

In [140]:
swissgrid_total = pd.concat([
    get_swissgrid_df(fp)
    for fp in (
        "data/EnergieUebersichtCH-2019.xls",
        "data/EnergieUebersichtCH-2020.xlsx",
        "data/EnergieUebersichtCH-2021.xlsx",
    )
])['kWh.1'] / 1000 # Convert to MWh

See notebook `Swiss electricity residual mix`

## Mapping ENTSO to ecoinvent

In [20]:
ENTSO_MAPPING = {
    'Biomass': [
        'heat and power co-generation, wood chips, 6667 kW',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
        'heat and power co-generation, biogas, gas engine',
    ],
    'Fossil Brown coal/Lignite': [
        'electricity production, lignite',
        'heat and power co-generation, lignite',
    ],
    'Fossil Coal-derived gas': [
        'treatment of coal gas, in power plant',
    ],
    'Fossil Gas': [
        'electricity production, natural gas, combined cycle power plant',
        'electricity production, natural gas, conventional power plant',
        'heat and power co-generation, natural gas, 500kW electrical, lean burn',
        'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
        'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
    ],
    'Fossil Hard coal': [
        'electricity production, hard coal',
        'heat and power co-generation, hard coal',
    ],
    'Fossil Oil': [
        'electricity production, oil',
        'heat and power co-generation, oil',
    ],
    'Fossil Oil shale': [
    ],
    'Fossil Peat': [
        'electricity production, peat',
    ],
    'Geothermal': [
        'electricity production, deep geothermal',
    ],
    # This will be a bit funky; could consider charging with average of last 48 hours?
    'Hydro Pumped Storage': [
        'electricity production, hydro, pumped storage',
    ],
    'Hydro Run-of-river and poundage': [
        'electricity production, hydro, run-of-river',
    ],
    'Hydro Water Reservoir': [
        'electricity production, hydro, reservoir, alpine region',
        'electricity production, hydro, reservoir, non-alpine region',
    ],
    'Nuclear': [
        'electricity production, nuclear, boiling water reactor',
        'electricity production, nuclear, pressure water reactor',
        'electricity production, nuclear, pressure water reactor, heavy water moderated',
    ],
    'Other': [
        'treatment of blast furnace gas, in power plant',
    ],
    'Other renewable': [
    ],
    'Solar': [
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
        'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 570kWp open ground installation, multi-Si',
        'electricity production, solar thermal parabolic trough, 50 MW',
        'electricity production, solar tower power plant, 20 MW',        
    ],
    'Waste': [
        'electricity, from municipal waste incineration to generic market for electricity, medium voltage',
    ],
    'Wind Offshore': [
        'electricity production, wind, 1-3MW turbine, offshore',
    ],
    'Wind Onshore': [
        'electricity production, wind, 1-3MW turbine, onshore',
        'electricity production, wind, <1MW turbine, onshore',
        'electricity production, wind, >3MW turbine, onshore',
    ],
}

In [21]:
set.union(*[set(v) for v in ENTSO_MAPPING.values()]).difference(all_generation)

set()

In [22]:
all_generation.difference(set.union(*[set(v) for v in ENTSO_MAPPING.values()]))

set()

In [38]:
mixes = {
    act['location']: act 
    for act in bd.Database("ecoinvent 3.8 cutoff") 
    if act['name'] == 'market for electricity, high voltage'
}

## Comparing aggregate values

In [23]:
from bentso import CachingDataClient as CDC

In [24]:
cdc = CDC()

Using data directory /Users/cmutel/Code/akula/entso-data-cache


In [25]:
def apply_country_specific_fixes(df, country):
    if country in ('GR', 'BG', 'ME'):
        df.rename(columns={'Hydro Water Reservoir': 'Hydro Run-of-river and poundage'}, inplace=True)
    elif country == 'NL':
        df.drop('Waste', axis=1, inplace=True)
    elif country == 'EE':
        df.rename(columns={'Fossil Oil shale': 'Fossil Oil'}, inplace=True)
    elif country == 'NO':
        df['Hydro Water Reservoir'] += df['Hydro Run-of-river and poundage']
        df.drop('Hydro Run-of-river and poundage', axis=1, inplace=True)
    elif country == 'RO':
        df['Hydro Run-of-river and poundage'] += df['Hydro Water Reservoir']
        df.drop('Hydro Water Reservoir', axis=1, inplace=True)
    elif country == 'BA':
        # Very wrong
        df['Fossil Brown coal/Lignite'] += df['Fossil Hard coal']
        df.drop('Fossil Hard coal', axis=1, inplace=True)        
    return df

In [26]:
def get_df(country, year):
    df = cdc.get_generation(country=country, year=year, clean=True, full_year=True, fix_lv=True)
    df = apply_country_specific_fixes(df, country)
    return df

Everything here is fine, we special case Switzerland and don't touch Malta or Luxembourg anyway.

In [ ]:
def check_missing_generators_small(country, year=2019):
    """Make sure that generators found in ecoinvent but missing in ENTSO are a small proportion of total generation"""
    found_missing = False

    markets = [get_one("ecoinvent 3.8 cutoff", name=name, location=country) for name in NAMES]
    generators = set.union(*[{
        exc.input['name'] 
        for act in markets for exc in act.technosphere() 
        if is_generation_exchange(exc)}
    ])
    df = get_df(country, year).sum(axis=0)
    fractions = df / df.sum()
    for label, amount in zip(fractions.index, fractions):
        if amount >= 0.01 and not any(generator in generators for generator in ENTSO_MAPPING[label]):
            if not found_missing:
                print("\n", "-" * 20, country, "-" * 20)
                print("Fractions:")
                for x, y in zip(fractions.index, fractions):
                    print(x, y)                
                print("\nGenerators:")
                for g in sorted(generators):
                    print(g)
                print()
                found_missing = True
            print("Missing:", label)  

In [28]:
for country in ENTSO_COUNTRIES:
    if country == 'CH':
        continue
    try:
        check_missing_generators_small(country)
    except NoMatchingDataError:
        print(f"No data for {country}")

KeyboardInterrupt: 

# Generating modified data for `ecoinvent` markets

For high voltage markets, we need to do the following:

* Zero out the `electricity, high voltage, import from XXX` activities
* Calculate the total generation plus imports
* Add exchanges with normalized trade from the high voltage market mixes of other countries
* Convert generation in ENTSO-E categories to normalized generation in ecoinvent activities

For medium and low voltage, we need to do the following:

* Calculate the ration of transformed versus voltage-specific generation
* Convert generation in ENTSO-E categories to normalized generation in ecoinvent activities
* Add an exchange with the normalized value from voltage transformation

In [153]:
class ENTSODataConverter:
    def __init__(self, ecoinvent_database):
        self.ei = bd.Database(ecoinvent_database)
        self.cdc = CDC()
        self.reverse_entso_mapping = {elem: k for k, v in ENTSO_MAPPING.items() for elem in v}
        
        # Prepare cache of activities
        self.hv_mixes = {
            act['location']: act 
            for act in self.ei
            if act['name'] == 'market for electricity, high voltage'
            and act['location'] in ENTSO_COUNTRIES
        }
        self.mv_mixes = {
            act['location']: act 
            for act in self.ei
            if act['name'] == 'market for electricity, medium voltage'
            and act['location'] in ENTSO_COUNTRIES
        }
        self.lv_mixes = {
            act['location']: act 
            for act in self.ei
            if act['name'] == 'market for electricity, low voltage'
            and act['location'] in ENTSO_COUNTRIES
        }
        self.mv_loss_coefficients = {
            act['location']: next(iter(act.technosphere()))['amount']
            for act in self.ei
            if act['name'] == 'electricity voltage transformation from high to medium voltage'
            and act['location'] in ENTSO_COUNTRIES
        }
        self.lv_loss_coefficients = {
            act['location']: next(iter(act.technosphere()))['amount']
            for act in self.ei
            if act['name'] == 'electricity voltage transformation from medium to low voltage'
            and act['location'] in ENTSO_COUNTRIES
        }  
    
    def get_ratio_hv_lv_solar_ecoinvent(self, country):
        """Get the fraction of high voltage solar generation in total solar generation as defined in ecoinvent markets"""
        hv_solar_in_low_voltage = (
            sum(
                exc['amount'] 
                for exc in self.hv_mixes[country].technosphere()
                if exc.input['name'] in ENTSO_MAPPING['Solar']
            ) 
            / self.mv_loss_coefficients[country] 
            / self.lv_loss_coefficients[country]
            * next(
                exc['amount'] 
                for exc in self.lv_mixes[country].technosphere()
                if exc.input['name'] == 'electricity voltage transformation from medium to low voltage'
            )
        )
        lv_solar_in_low_voltage = sum(
            exc['amount'] 
            for exc in self.lv_mixes[country].technosphere() 
            if exc.input['name'] in ENTSO_MAPPING['Solar']
        ) 
        denominator = (hv_solar_in_low_voltage + lv_solar_in_low_voltage) or 1
        return hv_solar_in_low_voltage / denominator

    def adjust_entso_df_for_low_voltage(self, df, country):
        """Adjust ENTSO dataframe for use in ecoinvent high voltage markets.

        1. Remove medium voltage waste treatment category.
        2. Adjust solar generation to only include high voltage generation.
        3. Convert all ENTSO categories to ecoinvent generators (i.e. disaggregate), using ecoinvent ratios.

        """
        market = self.lv_mixes[country]

        if 'Solar' not in df.columns:
            return (
                None,
                pd.Series(np.ones(df.shape[0]), index=df.index)
            )

        solar_lv = df['Solar'] * (1 - self.get_ratio_hv_lv_solar_ecoinvent(country))
        
        # This is now only high voltage
        df['Solar'] *= self.get_ratio_hv_lv_solar_ecoinvent(country)

        mv_generation_with_losses = (
            ((df.sum(axis=1) - df['Waste']) / self.mv_loss_coefficients[country] / self.lv_loss_coefficients[country])
            + (df['Waste'] / self.lv_loss_coefficients[country])
        )
        
        amounts = {
            exc.input['name']: exc['amount'] 
            for exc in market.technosphere() 
            if is_generation_exchange(exc) 
            and exc['amount']
        }    
        if not amounts:
            print("Warning with {}, average low voltage generation of {} but no ecoinvent exchanges".format(country, solar_lv.mean(axis=0)))
            return (
                None,
                pd.Series(np.ones(df.shape[0]), index=df.index)
            )
            
        subtotal = sum(amounts.values())
        disaggregated = pd.DataFrame(
            {name: amount / subtotal * solar_lv for name, amount in amounts.items()},
            index=df.index
        ).fillna(0)
        return disaggregated, mv_generation_with_losses
    
    def data_dict_for_low_voltage_market(self, country, years=(2019, 2020, 2021), average=False):
        data = {}

        market = self.lv_mixes[country]
        df, series = self.adjust_entso_df_for_low_voltage(pd.concat([get_df(country, year) for year in years]), country)

        imprt = next(
            exc.input 
            for exc in market.technosphere() 
            if exc.input['name'] == 'electricity voltage transformation from medium to low voltage'
        )
        
        if df is None:
            data[(imprt, market, True)] = series
        else:
            total = df.sum(axis=1) + series
            
            data[(imprt, market, True)] = series / total
            for exc in market.technosphere():
                if exc.input['name'] in df:
                    data[(exc.input, market, True)] = df[exc.input['name']] / total

        return data
    
    def adjust_entso_df_for_medium_voltage(self, df, country):
        """Adjust ENTSO dataframe for use in ecoinvent medium voltage markets.

        1. Get ratio of total hv generation (including T&D losses) to mv production
        2. Convert all ENTSO categories to ecoinvent generators (i.e. disaggregate), using ecoinvent ratios.
        
        """
        market = self.mv_mixes[country]
        
        if 'Waste' not in df.columns:
            return (
                pd.Series(np.zeros(df.shape[0]), index=df.index),
                pd.Series(np.ones(df.shape[0]), index=df.index)
            )
        if 'Solar' in df.columns:
            df['Solar'] *= self.get_ratio_hv_lv_solar_ecoinvent(country)

        hv_generation_with_losses = (df.sum(axis=1) - df['Waste']) / self.mv_loss_coefficients[country]
            
        if not any(
            exc['amount'] 
            for exc in market.technosphere() 
            if is_generation_exchange(exc) and exc.input['name'] in ENTSO_MAPPING['Waste']
        ):    
            print("Warning with {}, average waste generation of {} but no ecoinvent exchange".format(country, df['Waste'].mean(axis=0)))
            return (
                pd.Series(np.zeros(df.shape[0]), index=df.index),
                pd.Series(np.ones(df.shape[0]), index=df.index)
            )
        return df['Waste'].fillna(0), hv_generation_with_losses

    def data_dict_for_medium_voltage_market(self, country, years=(2019, 2020, 2021), average=False):
        data = {}

        market = self.mv_mixes[country]
        waste_series, remaining_series = self.adjust_entso_df_for_medium_voltage(
            pd.concat([get_df(country, year) for year in years]),
            country
        )
        
        imprt = next(
            exc.input 
            for exc in market.technosphere() 
            if exc.input['name'] == 'electricity voltage transformation from high to medium voltage'
        )
        try:
            waste_input = next(
                exc.input 
                for exc in market.technosphere() 
                if exc.input['name'] in ENTSO_MAPPING['Waste']
            )
        except StopIteration:
            waste_input = None
        
        total = waste_series + remaining_series
    
        data[(imprt, market, True)] = remaining_series / total
        if waste_input is not None:
            data[(waste_input, market, True)] = waste_series / total
        return data
    
    def adjust_entso_df_for_high_voltage(self, df, country):
        """Adjust ENTSO dataframe for use in ecoinvent high voltage markets.

        1. Remove medium voltage waste treatment category.
        2. Adjust solar generation to only include high voltage generation.
        3. Convert all ENTSO categories to ecoinvent generators (i.e. disaggregate), using ecoinvent ratios.

        """
        market = self.hv_mixes[country]
        
        if 'Waste' in df.columns:
            df.drop('Waste', axis=1, inplace=True)
        if 'Solar' in df.columns:
            df['Solar'] *= self.get_ratio_hv_lv_solar_ecoinvent(country)

        amounts = {exc.input['name']: exc['amount'] for exc in market.technosphere() if is_generation_exchange(exc)}    
        disaggregated = []

        for label, names in ENTSO_MAPPING.items():
            if label not in df.columns:
                continue
            inputs = {name: amounts[name] for name in names if amounts.get(name, 0)}
            if not inputs:
                continue
            subtotal = sum(inputs.values())
            disaggregated.append(pd.DataFrame(
                {name: amount / subtotal * df[label] for name, amount in inputs.items()},
                index=df.index
            ))
        return pd.concat(disaggregated, axis=1).fillna(0)
    
    def data_dict_for_high_voltage_market(self, country, years=(2019, 2020, 2021), average=False):
        data = {}
        
        market = self.hv_mixes[country]
        df = self.adjust_entso_df_for_high_voltage(pd.concat([get_df(country, year) for year in years]), country)
        empty = np.zeros((df.shape[0],))

        if country == 'CH':
            swissgrid_total_cut = swissgrid_total[:df.shape[0]]
            swissgrid_total_cut.index = df.index
            
        trade = defaultdict(list)
        for src in TRADE_PAIRS[country]:
            for year in years:
                try:
                    trade[src].append(cdc.get_trade(from_country=src, to_country=country, year=year, full_year=True))
                except:
                    pass
            if trade[src]:
                trade[src] = pd.concat(trade[src])
                assert trade[src].shape[0] == df.shape[0]

        if country == 'CH':
            total = swissgrid_total_cut + sum(trade.values())
        else:
            total = df.sum(axis=1) + sum(trade.values())
        
        for exc in market.technosphere():
            if exc.input['name'] in df:
                data[(exc.input, market.id, True)] = df[exc.input['name']] / total
            elif is_trade_exchange(exc) or is_generation_exchange(exc):
                data[(exc.input, market.id, False)] = total * 0
        for src, vector in trade.items():
            data[(mixes[src], market.id, True)] = vector / total

        if country == 'CH':
            data[(100000, market.id, True)] = (swissgrid_total_cut - df.sum(axis=1)) / total

        return data

In [154]:
edp = ENTSODataConverter("ecoinvent 3.8 cutoff")

Using data directory /Users/cmutel/Code/akula/entso-data-cache


In [161]:
data = edp.data_dict_for_high_voltage_market('CH')

In [162]:
for key, value in data.items():
    print(key[0], value.mean())
    
print(sum(value.mean() for value in data.values()))

'electricity, high voltage, biofuels, import from Germany' (kilowatt hour, CH, None) 0.0
'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, CH, None) 0.000287365729530342
'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, CH, None) 0.0
'electricity production, hydro, reservoir, alpine region' (kilowatt hour, CH, None) 0.1165436537503357
'electricity production, hydro, run-of-river' (kilowatt hour, CH, None) 0.021831878328883263
'electricity production, nuclear, boiling water reactor' (kilowatt hour, CH, None) 0.11310028539919396
'electricity production, nuclear, pressure water reactor' (kilowatt hour, CH, None) 0.12706562933340737
'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, CH, None) 0.0007612426562717657
'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, CH, None) 0.0001250497612282775
'electricity, high voltage, hydro, reservoir, import from France' (kilowatt hour, CH, None) 0.0
'

In [158]:
data = edp.data_dict_for_high_voltage_market('DE')

In [160]:
for key, value in data.items():
    print(key[0], value.mean())

print(sum(value.mean() for value in data.values()))

'electricity production, deep geothermal' (kilowatt hour, DE, None) 0.0004288200602826468
'electricity production, hard coal' (kilowatt hour, DE, None) 0.0799432871850347
'electricity production, hydro, pumped storage' (kilowatt hour, DE, None) 0.020125173662038014
'electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, DE, None) 0.002836875829660274
'electricity production, hydro, run-of-river' (kilowatt hour, DE, None) 0.032302656637288235
'electricity production, lignite' (kilowatt hour, DE, None) 0.20056857422614252
'electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None) 0.02369018835280738
'electricity production, natural gas, conventional power plant' (kilowatt hour, DE, None) 0.014632175170358261
'electricity production, nuclear, boiling water reactor' (kilowatt hour, DE, None) 0.031020245367810926
'electricity production, nuclear, pressure water reactor' (kilowatt hour, DE, None) 0.11452733917727524
'electricity producti

In [21]:
def compare_fractional_generation(country, year=2020):
    hv = ei.get(name='market for electricity, high voltage', location=country)
    mv = ei.get(name='market for electricity, medium voltage', location=country)
    lv = ei.get(name='market for electricity, low voltage', location=country)
    
    entso_df = get_df(country=country, year=2020).sum(axis=0)
    entso_df = (entso_df / entso_df.sum()).to_dict()
    
    sum_hv = sum(exc['amount'] for exc in hv.technosphere() if included_exchange(exc))
    # sum_mv = sum(exc['amount'] for exc in mv.technosphere() if included_exchange(exc))
    # sum_lv = sum(exc['amount'] for exc in lv.technosphere() if included_exchange(exc))
    conv_mv_to_lv = sum(exc['amount'] for exc in lv.technosphere() 
                        if exc.input['name'] == 'electricity voltage transformation from medium to low voltage')
    conv_hv_to_lv = (conv_mv_to_lv *
                     sum(exc['amount'] for exc in mv.technosphere() 
                         if exc.input['name'] == 'electricity voltage transformation from high to medium voltage')
                    )
    
    ei_data = {
        'Solar': sum(exc['amount'] for exc in lv.technosphere() if included_exchange(exc)),
        'Waste': sum(exc['amount'] for exc in mv.technosphere() if included_exchange(exc)) * conv_mv_to_lv,
    }
    
    for key, lst in ENTSO_MAPPING.items():
        if key in ei_data:
            continue
        exchanges = [exc for exc in hv.technosphere() if exc.input['name'] in lst]
        if not exchanges:
            continue
        
        ei_data[key] = (
            sum(exc['amount'] for exc in exchanges) 
            / sum_hv  # production doesn't add up to 1 - there are also imports
            * conv_hv_to_lv
        )
    
    all_keys = set(ei_data).union(set(entso_df))
    for key in all_keys:
        if key not in entso_df:
            entso_df[key] = 0
        if key not in ei_data:
            ei_data[key] = 0
    
    return pd.DataFrame({'ENTSO': entso_df, 'ecoinvent': ei_data})

In [29]:
(get_df('ES', 2019).sum(axis=0)).sum() / 1000

238652.11

In [31]:
df = compare_fractional_generation('ES', 2019)
df

,ENTSO,ecoinvent
Biomass,0.016056,0.012940
Fossil Brown coal/Lignite,0.000000,0.007119
Fossil Coal-derived gas,0.000000,0.000467
Fossil Gas,0.257964,0.120847
Fossil Hard coal,0.021526,0.147656
Fossil Oil,0.007724,0.040121
Fossil Oil shale,0.000000,0.000000
Fossil Peat,0.000000,0.000000
Geothermal,0.000000,0.000000
Hydro Run-of-river and poundage,0.042326,0.093979


Data read manually from https://www.iea.org/countries/spain

In [34]:
df['IEA'] = pd.Series({
    'Biomass': 4802,
    'Fossil Brown coal/Lignite': 0,
    'Fossil Coal-derived gas': 0,
    'Fossil Gas': 83703,
    'Fossil Hard coal': 13982,
    'Fossil Oil': 12883,
    'Fossil Oil shale': 0,
    'Fossil Peat': 0,
    'Geothermal': 0,
    'Hydro Run-of-river and poundage': 26874,
    'Hydro Water Reservoir': 0,
    'Marine': 0,
    'Nuclear': 58349,
    'Other': 139,
    'Other renewable': 0,
    'Solar': 9420,
    'Waste': 1755,
    'Wind Offshore': 0,
    'Wind Onshore': 55647,
    'Hydro Pumped Storage': 0,
})

In [35]:
df['IEA'] = df['IEA'] / df['IEA'].sum()

In [45]:
import requests

In [52]:
URL = "https://apidatos.ree.es/en/datos/generacion/estructura-generacion?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=year"

In [53]:
res = {o['attributes']['title']: o['attributes']['values'][0]['percentage'] for o in requests.get(URL).json()['included']}
res

{'Hydro': 0.09477104131458193,
 'Pumped storage': 0.00630875586852493,
 'Nuclear': 0.21402632991933143,
 'Coal': 0.04857837137561619,
 'Fuel + Gas': 3.8339327256957146e-12,
 'Diesel engines': 0.01087323696059382,
 'Gas turbine': 0.002570774873639019,
 'Steam turbine': 0.008392519636942236,
 'Combined cycle': 0.2117939974506982,
 'Hydroeolian': 8.913402077067102e-05,
 'Wind': 0.20797189501836943,
 'Solar photovoltaic': 0.03547161796662054,
 'Thermal solar': 0.01980774944186908,
 'Other renewables': 0.013870071713414273,
 'Cogeneration': 0.11354224357492992,
 'Non-renewable waste': 0.008520774804365222,
 'Renewable waste': 0.003411486055899025,
 'Total generation': 1}

In [58]:
res_mapping = {
    "Hydro": "Hydro Run-of-river and poundage",
    "Pumped storage": "Hydro Run-of-river and poundage",
    "Nuclear": "Nuclear",
    "Coal": "Fossil Hard coal",
    "Fuel + Gas": "Fossil Oil",
    "Diesel engines": "Fossil Oil",
    "Gas turbine": "Fossil Gas",
    "Steam turbine": "Fossil Gas",
    "Combined cycle": "Fossil Gas",
    "Hydroeolian": "Wind Onshore",
    "Wind": "Wind Onshore",
    "Solar photovoltaic": "Solar",
    "Thermal solar": "Solar",
    "Other renewables": "Other renewable",
    "Cogeneration": "Other",
    "Non-renewable waste": "Waste",
    "Renewable waste": "Waste",
}

In [62]:
res_series = {label: 0 for label in df.index}

for key, value in res.items():
    try:
        res_series[res_mapping[key]] += value
    except KeyError:
        print(key)
    
df["RES"] = pd.Series(res_series)

Total generation


In [63]:
df

,ENTSO,ecoinvent,IEA,RES
Biomass,0.016056,0.012940,0.017948,0.000000
Fossil Brown coal/Lignite,0.000000,0.007119,0.000000,0.000000
Fossil Coal-derived gas,0.000000,0.000467,0.000000,0.000000
Fossil Gas,0.257964,0.120847,0.312845,0.222757
Fossil Hard coal,0.021526,0.147656,0.052259,0.048578
Fossil Oil,0.007724,0.040121,0.048151,0.010873
Fossil Oil shale,0.000000,0.000000,0.000000,0.000000
Fossil Peat,0.000000,0.000000,0.000000,0.000000
Geothermal,0.000000,0.000000,0.000000,0.000000
Hydro Run-of-river and poundage,0.042326,0.093979,0.100443,0.101080


In [65]:
df.to_excel("Spain.xlsx")

In [93]:
from pandas import ExcelWriter

In [98]:
with pd.ExcelWriter("generation.xlsx") as writer:
    for country in ENTSO_COUNTRIES:
        df = compare_fractional_generation(country, 2019) 
        df['difference'] = df['ENTSO'] - df['ecoinvent']
        df.to_excel(writer, sheet_name=country)